Preparing

In [ ]:
import pandas
import numpy
import random

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, AvgPool1D, MaxPool1D, Activation, Reshape, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l1, l2

In [ ]:
#Get training data:
totalX = pandas.read_csv('totalX.csv')
totalX = totalX.apply(pandas.to_numeric, errors='coerce')
totalY = pandas.read_csv('totalY.csv')
totalY = totalY.apply(pandas.to_numeric, errors='coerce')

Create Model

In [ ]:
#1440 ->C5,2 668 ->A4 167 ->C4,1 164 ->A4 41 ->C6,1 36 ->A4 9 ->D9
model = Sequential()
model.add(Reshape(input_shape = (None, 1440), target_shape = (1440, 1)))
model.add(BatchNormalization())
model.add(Conv1D(filters = 2, kernel_size = 5, strides = 2, activation = 'relu', activity_regularizer = l2(0.0002)))
model.add(AvgPool1D(pool_size = 4))
model.add(Dropout(0.15))
model.add(BatchNormalization())
model.add(Conv1D(filters = 2, kernel_size = 4, strides = 1, activation = 'relu', activity_regularizer = l2(0.0002)))
model.add(AvgPool1D(pool_size = 4))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Conv1D(filters = 2, kernel_size = 6, strides = 1, activation = 'relu', activity_regularizer = l2(0.0002)))
model.add(AvgPool1D(pool_size = 4))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(9, activation = 'relu'))
model.add(Flatten())
model.add(Dense(2, activation = 'softmax'))
model.add(Reshape(input_shape = (None, 1, 2), target_shape = (2,)))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
#Setting checkpoints:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
mcp_save = ModelCheckpoint('best_wts.hdf5', save_best_only = True, monitor = 'val_loss', mode = 'min', save_freq = 'epoch')
#reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', mode = 'min', factor = 0.9, patience = 20, verbose = 1, min_delta = 1e-4)

#Helper to prepare row for prediction: 
def prep(df, row):
	return numpy.reshape(df.iloc()[row].to_numpy(), (1,1440))

In [ ]:
#Train:
model.fit(x = totalX, y = totalY, validation_split = 0.2, epochs = 1000, batch_size = 70, verbose = 1, callbacks = [mcp_save])

Epoch 1/1000
12/12 [==============================] - 2s 41ms/step - loss: 0.8539 - accuracy: 0.4843 - val_loss: 1.0363 - val_accuracy: 0.6117
Epoch 2/1000
12/12 [==============================] - 0s 10ms/step - loss: 0.7783 - accuracy: 0.5750 - val_loss: 0.8252 - val_accuracy: 0.6262
Epoch 3/1000
12/12 [==============================] - 0s 9ms/step - loss: 0.7584 - accuracy: 0.6115 - val_loss: 0.7575 - val_accuracy: 0.6262
Epoch 4/1000
12/12 [==============================] - 0s 10ms/step - loss: 0.7398 - accuracy: 0.6232 - val_loss: 0.7265 - val_accuracy: 0.6262
Epoch 5/1000
12/12 [==============================] - 0s 10ms/step - loss: 0.7166 - accuracy: 0.6213 - val_loss: 0.7087 - val_accuracy: 0.6262
Epoch 6/1000
12/12 [==============================] - 0s 10ms/step - loss: 0.7013 - accuracy: 0.6300 - val_loss: 0.6976 - val_accuracy: 0.6262
Epoch 7/1000
12/12 [==============================] - 0s 10ms/step - loss: 0.6925 - accuracy: 0.6485 - val_loss: 0.6900 - val_accuracy: 0.6262


In [ ]:
#Evaluate model:
model.evaluate(totalX, totalY)

33/33 [==============================] - 0s 3ms/step - loss: 0.4305 - accuracy: 0.8047


[0.4304749071598053, 0.8046647310256958]

In [ ]:
model = keras.models.load_model('best_wts.hdf5')